In [1]:
from collections import OrderedDict, deque, namedtuple
from typing import Dict, List, Tuple
import numpy as np
import gym
# Torch imports 
import torch as th
from torch import Tensor, nn

# Torch lightning imports
from pytorch_lightning import LightningModule, Trainer, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint

/usr/local/lib/python3.9/dist-packages/torchmetrics/utilities/imports.py:24: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _PYTHON_LOWER_3_8 = LooseVersion(_PYTHON_VERSION) < LooseVersion("3.8")
/usr/local/lib/python3.9/dist-packages/torchvision/transforms/functional_pil.py:228: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  interpolation: int = Image.BILINEAR,
/usr/local/lib/python3.9/dist-packages/torchvision/transforms/functional_pil.py:295: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  interpolation: int = Image.NEAREST,
/usr/local/lib/python3.9/dist-packages/torchvision/transforms/functional_pil.py:328: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  interpolation: int = Image.BICUBIC,


In [1]:

from torch.optim import Adam, Optimizer
from torch.utils.data import DataLoader
from torch.utils.data.dataset import IterableDataset

# Torch lightning imports
from pytorch_lightning import LightningModule, Trainer, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint
#from pytorch_lightning.loggers import CSVLogger

# Common submodule imports
from common.replay_buffers import ReplayBuffer
from common.networks import MLP, GaussianPolicy
from common.torch_data_handlers import Transition, RLDataset
from common.noise import OrnsteinUhlenbeckActionNoise

#th.set_default_dtype(th.float64)

/usr/local/lib/python3.9/dist-packages/torchmetrics/utilities/imports.py:24: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _PYTHON_LOWER_3_8 = LooseVersion(_PYTHON_VERSION) < LooseVersion("3.8")
/usr/local/lib/python3.9/dist-packages/torchvision/transforms/functional_pil.py:228: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  interpolation: int = Image.BILINEAR,
/usr/local/lib/python3.9/dist-packages/torchvision/transforms/functional_pil.py:295: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  interpolation: int = Image.NEAREST,
/usr/local/lib/python3.9/dist-packages/torchvision/transforms/functional_pil.py:328: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  interpolation: int = Image.BICUBIC,


# I - Original DDPG

In [6]:
#from pl_ddpg import DDPG
from DDPG import DDPG

In [ ]:
del model

In [7]:
# Instantiate DDPG model
model = DDPG(env = gym.make("HalfCheetah-v4"),
             n_warm_start_steps=10000, 
             num_workers = 6,
            )


/usr/local/lib/python3.9/dist-packages/gym/utils/env_checker.py:144: UserWarning: WARN: Agent's minimum observation space value is -infinity. This is probably too low.
  logger.warn(
/usr/local/lib/python3.9/dist-packages/gym/utils/env_checker.py:148: UserWarning: WARN: Agent's maxmimum observation space value is infinity. This is probably too high
  logger.warn(


In [4]:
'''
    #####################################
    ### TRAINER WITH MODEL CHECKPOINT ###
    #####################################
'''

# save checkpoints based on avg_reward
checkpoint_callback = ModelCheckpoint(save_top_k=1, 
                                      monitor="avg_score", 
                                      mode="max", 
                                      verbose=True)
seed_everything(123)
trainer = Trainer(accelerator="gpu", 
                  max_steps=500000,  # 100000, 1000000
                  callbacks = checkpoint_callback)

Global seed set to 123
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
del checkpoint_callback, trainer

In [8]:
'''
    ##########################################
    ### TEST TRAINER (NO MODEL CHECKPOINT) ###
    ##########################################
'''

seed_everything(123)
trainer = Trainer(accelerator="gpu",
                  max_steps=200,)

Global seed set to 123
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [9]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type | Params
--------------------------------------
0 | q_net        | MLP  | 72.2 K
1 | target_q_net | MLP  | 72.2 K
2 | policy       | MLP  | 71.9 K
--------------------------------------
216 K     Trainable params
0         Non-trainable params
216 K     Total params
0.865     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:234: UserWarning: You called `self.log('episodes', ...)` in your `training_step` but the value needs to be floating point. Converting it to torch.float32.
  warning_cache.warn(
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:234: UserWarning: You called `self.log('episode_steps', ...)` in your `training_step` but the value needs to be floating point. Converting it to torch.float32.
  warning_cache.warn(
`Trainer.fit` stopped: `max_steps=200` reached.


In [ ]:
model.device

____________________________________________________________________________________

### Recorded test

In [6]:
from gym import wrappers
from pyvirtualdisplay import Display

In [ ]:
del test_model

In [ ]:
'''
    ########################
    ### LOAD SAVED MODEL ###
    ########################
'''
# Last saved model
# test_model = DDPGS(env = gym.make("HalfCheetah-v4", render_mode = "rgb_array"))
test_model = DDPGS(env = gym.make("HalfCheetah-v4"))
test_model.load_state_dict(torch.load("./pl_ddpg_gym0240_2306272205"))

In [8]:
'''
    ############################
    ### LOAD FROM CHECKPOINT ###
    ############################
'''

# Best model from Trainer checkpoint
#test_model = DDPGS(env = gym.make("HalfCheetah-v4", render_mode = "rgb_array"))
test_model = DDPG.load_from_checkpoint("./lightning_logs/version_28/checkpoints/epoch=24-step=500000.ckpt")

# disable randomness, dropout, etc...
test_model.eval()

DDPG(
  (q_net): MLP(
    (net): Sequential(
      (0): Linear(in_features=23, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): ReLU()
      (4): Linear(in_features=256, out_features=1, bias=True)
    )
  )
  (target_q_net): MLP(
    (net): Sequential(
      (0): Linear(in_features=23, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): ReLU()
      (4): Linear(in_features=256, out_features=1, bias=True)
    )
  )
  (policy): MLP(
    (net): Sequential(
      (0): Linear(in_features=17, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): ReLU()
      (4): Linear(in_features=256, out_features=6, bias=True)
      (5): Tanh()
    )
  )
)

In [ ]:
'''
    ##############################
    ### USE LAST TRAINED MODEL ###
    ##############################
'''
test_model = model

test_model.eval()

In [ ]:
test_model.cpu()

In [9]:
# Make virtual display
virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

# Trigger for wrapper.RecordVideo() object
def epsd_trigger(episode_id: int) -> bool:
    '''
        Records all episodes
    '''
    if episode_id < 10:
        return True
    else:
        return False

In [10]:
del env

In [10]:
# Test env
#env = gym.make("HalfCheetah-v4", render_mode = "rgb_array")
env = gym.make("HalfCheetah-v4")
env = wrappers.RecordVideo(env = env, 
                           video_folder="vids/",
                           name_prefix="myDDPG_pl_gym0240_500kSteps_2307022345",
                           episode_trigger = epsd_trigger)

/usr/local/lib/python3.9/dist-packages/gym/utils/env_checker.py:144: UserWarning: WARN: Agent's minimum observation space value is -infinity. This is probably too low.
  logger.warn(
/usr/local/lib/python3.9/dist-packages/gym/utils/env_checker.py:148: UserWarning: WARN: Agent's maxmimum observation space value is infinity. This is probably too high
  logger.warn(
/usr/local/lib/python3.9/dist-packages/gym/wrappers/record_video.py:75: UserWarning: WARN: Overwriting existing videos at /notebooks/Lightning/vids folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [11]:
# Main loop
for episode in range(3):
    
    # Init. env. and counters
    state = env.reset()
    step = 0
    total_reward = 0
    n_prds = 0
    done = False
    trunc = False
    
    # Episodic loop
    while not done and step<5001:
        
        # Render
        env.render()
        
        # Get action
        with torch.no_grad():
            state_ = torch.FloatTensor(np.array(state)).to(test_model.device)
            # Get actions and UPolicy output
            #action = test_model.get_action(state_, test_model.device)
            action_ = test_model.policy(state_)
            # Get np arrays
            action = action_.cpu().detach().numpy()
        
        # Environment step
        state_next, reward, done, info = env.step(action)
        # Update tot. score
        total_reward += reward
        # Update step
        step += 1
        # Update state
        state = state_next
        # End of episode
        if done:
            print(f"Episode: {episode},\tSteps: {step},\tscore: {total_reward}")
            break

# Close environment
env.close()

Episode: 0,	Steps: 1000,	score: 3268.8982709639536
Episode: 1,	Steps: 1000,	score: 3148.7870928869233
Episode: 2,	Steps: 1000,	score: 2503.5117841006786


In [ ]:
test_model.device

# II - DDPG with a Gaussian policy

In [2]:
from pl_gausspol_ddpg import GaussPolicy_DDPG

In [ ]:
del model

In [3]:
# Instantiate DDPG model
model = GaussPolicy_DDPG(env = gym.make("HalfCheetah-v4"),
                         n_warm_start_steps=10000,
                         num_workers = 6,
                        )


/usr/local/lib/python3.9/dist-packages/gym/utils/env_checker.py:144: UserWarning: WARN: Agent's minimum observation space value is -infinity. This is probably too low.
  logger.warn(
/usr/local/lib/python3.9/dist-packages/gym/utils/env_checker.py:148: UserWarning: WARN: Agent's maxmimum observation space value is infinity. This is probably too high
  logger.warn(


In [4]:
'''
    #####################################
    ### TRAINER WITH MODEL CHECKPOINT ###
    #####################################
'''

# save checkpoints based on avg_reward
checkpoint_callback = ModelCheckpoint(save_top_k=1, 
                                      monitor="avg_score", 
                                      mode="max", 
                                      verbose=True)
seed_everything(123)
trainer = Trainer(accelerator="gpu", 
                  max_steps=200000,  # 100000, 1000000
                  callbacks = checkpoint_callback)

Global seed set to 123
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
del checkpoint_callback, trainer

In [4]:
'''
    ##########################################
    ### TEST TRAINER (NO MODEL CHECKPOINT) ###
    ##########################################
'''

seed_everything(123)
trainer = Trainer(accelerator="gpu",
                  max_steps=10000,)

Global seed set to 123
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [5]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type           | Params
------------------------------------------------
0 | q_net        | MLP            | 72.2 K
1 | target_q_net | MLP            | 72.2 K
2 | policy       | GaussianPolicy | 73.5 K
------------------------------------------------
217 K     Trainable params
0         Non-trainable params
217 K     Total params
0.871     Total estimated model params size (MB)
/usr/local/lib/python3.9/dist-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, '__version__') or LooseVersion(tensorboard.__version__) < LooseVersion('1.15'):
/usr/local/lib/python3.9/dist-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/usr/local/lib/python3.9/dist-packages/keras/utils/image_utils.py:36: Dep

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/torch/utils/data/_utils/collate.py:148: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return torch.as_tensor(batch)
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:234: UserWarning: You called `self.log('episodes', ...)` in your `training_step` but the value needs to be floating point. Converting it to torch.float32.
  warning_cache.warn(
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:234: UserWarning: You called `self.log('episode_steps', ...)` in your `training_step` but the value needs to be floating point. Converting it to torch.float32.
  warning_cache.warn(
Epoch 0, global step 20000: 'avg_score' reached -58.34665 (best -58.34665), saving model to '/notebooks/Lightning/lightning_logs/version_7/checkpoints/epoch=0-step=20000.ckpt' as top 1
Epoch 1, global step 40000: 'avg_sc

In [ ]:
model.device

____________________________________________________________________________________

### Recorded test

In [6]:
from gym import wrappers
from pyvirtualdisplay import Display

In [ ]:
del test_model

In [ ]:
'''
    ########################
    ### LOAD SAVED MODEL ###
    ########################
'''
# Last saved model
# test_model = DDPGS(env = gym.make("HalfCheetah-v4", render_mode = "rgb_array"))
test_model = DDPGS(env = gym.make("HalfCheetah-v4"))
test_model.load_state_dict(torch.load("./pl_ddpg_gym0240_2306272205"))

In [7]:
'''
    ############################
    ### LOAD FROM CHECKPOINT ###
    ############################
'''

# Best model from Trainer checkpoint
#test_model = DDPGS(env = gym.make("HalfCheetah-v4", render_mode = "rgb_array"))
test_model = GaussPolicy_DDPG.load_from_checkpoint("./lightning_logs/version_7/checkpoints/epoch=9-step=200000.ckpt")

# disable randomness, dropout, etc...
test_model.eval()

GaussPolicy_DDPG(
  (q_net): MLP(
    (net): Sequential(
      (0): Linear(in_features=23, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): ReLU()
      (4): Linear(in_features=256, out_features=1, bias=True)
    )
  )
  (target_q_net): MLP(
    (net): Sequential(
      (0): Linear(in_features=23, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): ReLU()
      (4): Linear(in_features=256, out_features=1, bias=True)
    )
  )
  (policy): GaussianPolicy(
    (feature_extractor): Sequential(
      (0): Linear(in_features=17, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): ReLU()
    )
    (mu_out_layer): Linear(in_features=256, out_features=6, bias=True)
    (log_sigma_out_layer): Linear(in_features=256, out_features=6, bias=True)
  )
)

In [ ]:
'''
    ##############################
    ### USE LAST TRAINED MODEL ###
    ##############################
'''
test_model = model

test_model.eval()

In [ ]:
test_model.cpu()

In [8]:
# Make virtual display
virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

# Trigger for wrapper.RecordVideo() object
def epsd_trigger(episode_id: int) -> bool:
    '''
        Records all episodes
    '''
    if episode_id < 10:
        return True
    else:
        return False

In [ ]:
del env

In [9]:
# Test env
#env = gym.make("HalfCheetah-v4", render_mode = "rgb_array")
env = gym.make("HalfCheetah-v4")
env = wrappers.RecordVideo(env = env, 
                           video_folder="vids/",
                           name_prefix="GaussPolDDPG_pl_gym0240_200kSteps_2307052200",
                           episode_trigger = epsd_trigger)

/usr/local/lib/python3.9/dist-packages/gym/utils/env_checker.py:144: UserWarning: WARN: Agent's minimum observation space value is -infinity. This is probably too low.
  logger.warn(
/usr/local/lib/python3.9/dist-packages/gym/utils/env_checker.py:148: UserWarning: WARN: Agent's maxmimum observation space value is infinity. This is probably too high
  logger.warn(
/usr/local/lib/python3.9/dist-packages/gym/wrappers/record_video.py:75: UserWarning: WARN: Overwriting existing videos at /notebooks/Lightning/vids folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [10]:
# Main loop
for episode in range(3):
    
    # Init. env. and counters
    state = env.reset()
    step = 0
    total_reward = 0
    n_prds = 0
    done = False
    trunc = False
    
    # Episodic loop
    while not done and step<5001:
        
        # Render
        env.render()
        
        # Get action
        with torch.no_grad():
            state_ = torch.FloatTensor(np.array(state)).to(test_model.device)
            # Get actions and UPolicy output
            #action = test_model.get_action(state_, test_model.device)
            action_ = test_model.policy.forward_mu(state_)
            # Get np arrays
            action = action_.cpu().detach().numpy()
        
        # Environment step
        state_next, reward, done, info = env.step(action)
        # Update tot. score
        total_reward += reward
        # Update step
        step += 1
        # Update state
        state = state_next
        # End of episode
        if done:
            print(f"Episode: {episode},\tSteps: {step},\tscore: {total_reward}")
            break

# Close environment
env.close()

Episode: 0,	Steps: 1000,	score: -322204.3222067865
Episode: 1,	Steps: 1000,	score: -315308.709114402
Episode: 2,	Steps: 1000,	score: -311572.3465534488


# III - DDPG with Generalized State-Dependent Exploration

In [1]:
from collections import OrderedDict, deque, namedtuple
from typing import Dict, List, Tuple
import numpy as np
import gym
# Torch imports 
import torch as th
from torch import Tensor, nn

# Torch lightning imports
from pytorch_lightning import LightningModule, Trainer, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint


from DDPG import GSDE_DDPG

/usr/local/lib/python3.9/dist-packages/torchmetrics/utilities/imports.py:24: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _PYTHON_LOWER_3_8 = LooseVersion(_PYTHON_VERSION) < LooseVersion("3.8")
/usr/local/lib/python3.9/dist-packages/torchvision/transforms/functional_pil.py:228: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  interpolation: int = Image.BILINEAR,
/usr/local/lib/python3.9/dist-packages/torchvision/transforms/functional_pil.py:295: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  interpolation: int = Image.NEAREST,
/usr/local/lib/python3.9/dist-packages/torchvision/transforms/functional_pil.py:328: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  interpolation: int = Image.BICUBIC,


In [ ]:
del model

In [2]:
# Instantiate DDPG model
model = GSDE_DDPG(env = gym.make("HalfCheetah-v4"),
                         n_warm_start_steps=10000,#10000,
                         num_workers = 6,
                        )


/usr/local/lib/python3.9/dist-packages/gym/utils/env_checker.py:144: UserWarning: WARN: Agent's minimum observation space value is -infinity. This is probably too low.
  logger.warn(
/usr/local/lib/python3.9/dist-packages/gym/utils/env_checker.py:148: UserWarning: WARN: Agent's maxmimum observation space value is infinity. This is probably too high
  logger.warn(


In [3]:
'''
    #####################################
    ### TRAINER WITH MODEL CHECKPOINT ###
    #####################################
'''

# save checkpoints based on avg_reward
checkpoint_callback = ModelCheckpoint(save_top_k=1, 
                                      monitor="avg_score", 
                                      mode="max", 
                                      verbose=True)
seed_everything(123)
trainer = Trainer(accelerator="gpu", 
                  max_steps=500000,  # 100000, 1000000
                  callbacks = checkpoint_callback)

Global seed set to 123
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
del checkpoint_callback, trainer

In [3]:
'''
    ##########################################
    ### TEST TRAINER (NO MODEL CHECKPOINT) ###
    ##########################################
'''

seed_everything(123)
trainer = Trainer(accelerator="gpu",
                  max_steps=10000,)

Global seed set to 123
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [4]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type       | Params
--------------------------------------------
0 | q_net        | MLP        | 72.2 K
1 | target_q_net | MLP        | 72.2 K
2 | policy       | GSDEPolicy | 73.5 K
--------------------------------------------
217 K     Trainable params
0         Non-trainable params
217 K     Total params
0.871     Total estimated model params size (MB)
/usr/local/lib/python3.9/dist-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, '__version__') or LooseVersion(tensorboard.__version__) < LooseVersion('1.15'):
/usr/local/lib/python3.9/dist-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/usr/local/lib/python3.9/dist-packages/keras/utils/image_utils.py:36: DeprecationWarning: NEAREST

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/torch/utils/data/_utils/collate.py:148: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return torch.as_tensor(batch)
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:234: UserWarning: You called `self.log('episodes', ...)` in your `training_step` but the value needs to be floating point. Converting it to torch.float32.
  warning_cache.warn(
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:234: UserWarning: You called `self.log('episode_steps', ...)` in your `training_step` but the value needs to be floating point. Converting it to torch.float32.
  warning_cache.warn(
Epoch 0, global step 20000: 'avg_score' reached -35.61629 (best -35.61629), saving model to '/notebooks/Lightning/lightning_logs/version_9/checkpoints/epoch=0-step=20000.ckpt' as top 1
Epoch 1, global step 40000: 'avg_sc

____________________________________________________________________________________

### Recorded test

In [4]:
from gym import wrappers
from pyvirtualdisplay import Display

In [ ]:
del test_model

In [ ]:
'''
    ########################
    ### LOAD SAVED MODEL ###
    ########################
'''
# Last saved model
# test_model = DDPGS(env = gym.make("HalfCheetah-v4", render_mode = "rgb_array"))
test_model = DDPGS(env = gym.make("HalfCheetah-v4"))
test_model.load_state_dict(torch.load("./pl_ddpg_gym0240_2306272205"))

In [2]:
'''
    ############################
    ### LOAD FROM CHECKPOINT ###
    ############################
'''

# Best model from Trainer checkpoint
#test_model = DDPGS(env = gym.make("HalfCheetah-v4", render_mode = "rgb_array"))
test_model = GSDE_DDPG.load_from_checkpoint("./lightning_logs/version_09_gSDEDDPG_500kSteps/checkpoints/epoch=24-step=500000.ckpt")

# disable randomness, dropout, etc...
test_model.eval()

GSDE_DDPG(
  (q_net): MLP(
    (net): Sequential(
      (0): Linear(in_features=23, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): ReLU()
      (4): Linear(in_features=256, out_features=1, bias=True)
    )
  )
  (target_q_net): MLP(
    (net): Sequential(
      (0): Linear(in_features=23, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): ReLU()
      (4): Linear(in_features=256, out_features=1, bias=True)
    )
  )
  (policy): GSDEPolicy(
    (feature_extractor): Sequential(
      (0): Linear(in_features=17, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): ReLU()
    )
    (mu_out_layer): Linear(in_features=256, out_features=6, bias=True)
    (output_activation_fn): Tanh()
  )
)

In [ ]:
'''
    ##############################
    ### USE LAST TRAINED MODEL ###
    ##############################
'''
test_model = model

test_model.eval()

In [ ]:
test_model.cpu()

In [5]:
# Make virtual display
virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

# Trigger for wrapper.RecordVideo() object
def epsd_trigger(episode_id: int) -> bool:
    '''
        Records all episodes
    '''
    if episode_id < 10:
        return True
    else:
        return False

In [ ]:
del env

In [6]:
# Test env
#env = gym.make("HalfCheetah-v4", render_mode = "rgb_array")
env = gym.make("HalfCheetah-v4")
env = wrappers.RecordVideo(env = env, 
                           video_folder="vids/",
                           name_prefix="gSDEDDPG_pl_gym0240_500kStepsDetrmc_2307101855",
                           episode_trigger = epsd_trigger)

/usr/local/lib/python3.9/dist-packages/gym/utils/env_checker.py:144: UserWarning: WARN: Agent's minimum observation space value is -infinity. This is probably too low.
  logger.warn(
/usr/local/lib/python3.9/dist-packages/gym/utils/env_checker.py:148: UserWarning: WARN: Agent's maxmimum observation space value is infinity. This is probably too high
  logger.warn(
/usr/local/lib/python3.9/dist-packages/gym/wrappers/record_video.py:75: UserWarning: WARN: Overwriting existing videos at /notebooks/Lightning/vids folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [7]:
test_model.policy.set_random_weights_distribution()
test_model.policy.sample_random_weights()

# Main loop
for episode in range(3):
    
    # Init. env. and counters
    state = env.reset()
    step = 0
    total_reward = 0
    n_prds = 0
    done = False
    trunc = False
    
    # Episodic loop
    while not done and step<5001:
        
        # Render
        env.render()
        
        # Get action
        with th.no_grad():
            state_ = th.FloatTensor(np.array(state)).to(test_model.device)
            
            #action = test_model.get_action(state_, test_model.device, deterministic=False)
            mu_ = test_model.policy.forward_mu(state_)
            action_ = test_model.policy.output_activation_fn(mu_)
            # Get np arrays
            action = action_.cpu().detach().numpy()
        
        # Environment step
        state_next, reward, done, info = env.step(action)
        # Update tot. score
        total_reward += reward
        # Update step
        step += 1
        # Update state
        state = state_next
        # End of episode
        if done:
            print(f"Episode: {episode},\tSteps: {step},\tscore: {total_reward}")
            break

# Close environment
env.close()

Episode: 0,	Steps: 1000,	score: 3184.802152807015
Episode: 1,	Steps: 1000,	score: 2567.0954478574845
Episode: 2,	Steps: 1000,	score: 2892.501496492099


________________________________________________________________________

### Scrap

In [ ]:
POLICY_ARCH = {"net_arch":[256, 256],
                       "dropout_probs":[],
                       "layer_activation_fn":nn.ReLU,
                       "output_activation_fn":nn.Tanh,
                       "weight_init_mthd": "",
                       "weight_init_seed":123,}

CRITIC_ARCH = {"net_arch":[256, 256],
                       "dropout_probs":[],
                       "layer_activation_fn":nn.ReLU,
                       "output_activation_fn":nn.Identity,
                       "weight_init_mthd": "",
                       "weight_init_seed":123,}

model = DDPG(env = gym.make("HalfCheetah-v4"),
             policy_arch_dict=POLICY_ARCH,
             q_net_arch_dict=CRITIC_ARCH,
             n_warm_start_steps=5000,
            )

In [ ]:
# save checkpoints based on avg_reward
checkpoint_callback = ModelCheckpoint(save_top_k=1, 
                                      monitor="avg_score", 
                                      mode="max", 
                                      verbose=True)
seed_everything(123)
trainer = Trainer(accelerator="gpu", 
                  max_steps=20000,  # 100000, 1000000
                  callbacks = checkpoint_callback)

In [ ]:
trainer.fit(model)